# Manipulation.py: Pick and Place example

In [1]:
import numpy as np
import robotic as ry
import manipulation as manip

In [2]:
C = ry.Config()
C.addFile(ry.raiPath('../rai-robotModels/scenarios/pandaSingle.g'))

midpoint = np.array([-0.105, 0.4, 0.705-.025+.1])
C.addFrame("box") \
    .setPosition(midpoint) \
    .setShape(ry.ST.ssBox, size=[0.06, 0.12, 0.06, 0.001]) \
    .setColor([1, 0, 0]) \
    .setContact(1) \
    .setMass(.1)

# For convenience, a few definitions:
gripper = "l_gripper"
box = "box"
table = "table"

C.view()

0

In [3]:
bot = ry.BotOp(C, useRealRobot=False)
bot.home(C)
bot.gripperMove(ry._left)

In [4]:
def pick_place(obj: str, grasp_direction: str, place_direction: str, place_position: list[float], info: str="", vis: bool=False) -> bool:
	M = manip.ManipulationModelling(C, info, helpers=[gripper])
	M.setup_pick_and_place_waypoints(gripper, obj, homing_scale=1e-1)
	M.grasp_box(1., gripper, obj, "l_palm", grasp_direction)
	M.place_box(2., obj, table, "l_palm", place_direction)
	M.target_relative_xy_position(2., obj, table, place_position)
	M.solve()
	if not M.feasible:
		return False, [], []

	M1 = M.sub_motion(0)
	M1.keep_distance([.3,.7], "l_palm", obj, margin=.05)
	M1.retract([.0, .2], gripper)
	M1.approach([.8, 1.], gripper)
	path1 = M1.solve()
	if not M1.feasible:
		return False, [], []

	M2 = M.sub_motion(1)
	M2.keep_distance([], table, "panda_collCameraWrist")
	M2.keep_distance([.2, .8], table, obj, .04)
	M2.keep_distance([], "l_palm", obj)
	path2 = M2.solve()
	if not M2.feasible:
		return False, [], []

	if vis:
		M1.play(C, 1.)
		C.attach(gripper, obj)
		M2.play(C, 1.)
		C.attach(table, obj)

	return True, path1, path2

In [5]:
attempt_count = 100
success_count = 0

for l in range(attempt_count):

	target_position = [
		midpoint[0] + np.random.uniform(-.15, .15),
		midpoint[1] + np.random.uniform(-.15, .15),
		0.]
	
	grasp_direction = np.random.choice(["x", "z"])  # "y" not possible: box too large
	place_direction = np.random.choice(["x", "y", "z", "xNeg", "yNeg", "zNeg"])

	success, path1, path2 = pick_place(box, grasp_direction, place_direction, target_position, str(l), vis=True)

	if success:
		bot.move(path1, [3])
		while bot.getTimeToEnd() > 0.:
			bot.sync(C)
		bot.gripperClose(ry._left)
		while not bot.gripperDone(ry._left):
			bot.sync(C)
		bot.move(path2, [3])
		while bot.getTimeToEnd() > 0.:
			bot.sync(C)
		bot.gripperMove(ry._left)
		while not bot.gripperDone(ry._left):
			bot.sync(C)
		
	success_count += 1 if success else 0
	
print(f"Successful motions: {success_count}/{attempt_count}")

  -- feasible:0
     { time: 0.145648, evals: 200, done: 1, feasible: 1, sos: 0.279742, f: 0, ineq: 0.0130856, eq: 0.46153 }
  -- infeasible:sub_motion_0--0
     { time: 0.535132, evals: 201, done: 1, feasible: 0, sos: 63.6724, f: 0, ineq: 0.00292133, eq: 1.00778 }
  -- infeasible:1
     { time: 0.100432, evals: 200, done: 1, feasible: 0, sos: 0.220718, f: 0, ineq: 0.215775, eq: 1.20614 }
  -- feasible:2
     { time: 0.102486, evals: 201, done: 1, feasible: 1, sos: 0.301228, f: 0, ineq: 0.0210772, eq: 0.44881 }
  -- infeasible:sub_motion_0--2
     { time: 0.302416, evals: 112, done: 1, feasible: 0, sos: 7.5041, f: 0, ineq: 0, eq: 1.70127 }
  -- infeasible:3
     { time: 0.114944, evals: 200, done: 1, feasible: 0, sos: 0.274281, f: 0, ineq: 0.0124837, eq: 0.500012 }
  -- feasible:4
     { time: 0.054934, evals: 105, done: 1, feasible: 1, sos: 0.108352, f: 0, ineq: 0.0029426, eq: 0.0349575 }
  -- feasible:sub_motion_0--4
     { time: 0.198137, evals: 95, done: 1, feasible: 1, sos: 2.0882

KeyboardInterrupt: 